In [ ]:
import glob#Return all file paths which match a specific pattern 
import os#Operating system interface
import numpy as np#Numerical library
import pandas as pd#Dataframe library
import matplotlib.pyplot as plt#Plotting library


import seaborn as sns#Plotting library
import tensorflow as tf#Machine learning library
from tensorflow import keras#Keras a high level API for TensorFlow
from tensorflow.keras.applications import vgg16#VGG16 model from Keras
from tensorflow.keras.applications import resnet50#ResNet50 model from Keras
from tensorflow.keras.applications import inception_v3#InceptionV3 model from Keras
from tensorflow.keras.applications import efficientnet#EfficientNet model from Keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator# Image preprocessing

In [ ]:
path = "C:/Users/tedjo/Downloads/XRAY/SplitData"#Path to the data

In [ ]:
#Train Path to dataset
train_path = os.path.join(path, 'train')
#Test Path to dataset
test_path = os.path.join(path, 'test')
#Validation Path to dataset
val_path = os.path.join(path, 'val')

In [ ]:
#COVID 19 Path to images
c_train_images = glob.glob(f'{train_path}/COVID/*.jpg')
#Normal Path to images
n_train_images = glob.glob(f'{train_path}/NORMAL/*.jpg')
#Pneumonia Path to images
p_train_images = glob.glob(f'{train_path}/PNEUMONIA/*.jpg')

In [ ]:
#Concatenate the number of images for each class 
#Place them in to a dataframe array
dataFrame = pd.DataFrame(np.concatenate([[0]*len(c_train_images), [1]*len(n_train_images), [2]*len(p_train_images)]), columns=['Label'])

In [ ]:
#plot the count 
ax = sns.countplot(dataFrame['Label'],data=dataFrame)
#label the number of data in each label
abs_counts = dataFrame['Label'].value_counts()
rel_values = dataFrame['Label'].value_counts(normalize=True).values
#0 represents COVID-19
#1 represents Normal
#2 represents Pneumonia

In [ ]:
#Drawing a piechart to presents the percentage of data
#0 represents COVID-19
#1 represents Normal
#2 represents Pneumonia
labels = [f'{p[0]} ({p[1]:.2f}%)' for p in zip(abs_counts.index, rel_values*100)]
plt.pie(rel_values, labels=labels)

In [ ]:
#VGG16 model
model16 = vgg16.VGG16(weights='imagenet', include_top=False,input_shape=(64,64,3))

#The layers in this layers are frozen meaning that the layer will not be trained
#This is will prevent the weight to be modified
for i in model16.layers:
    i.trainable = False

output = model16.output
#The Average Pooling Layer
output = keras.layers.GlobalAveragePooling2D()(output)#Pooling operation for spatial data(Data that directly or indirectly references spatial information)
output = keras.layers.Dense(128, activation='relu')(output)#Dense indicates the number of neurons in the layer, 128 Neurons 
#The classification Layer
predictions = keras.layers.Dense(12, activation='softmax')(output)#12 Neurons 
#The final Model
model16 = keras.models.Model(inputs=model16.input, outputs=predictions)#Group the layers into an object with training and interface features
#Learning rate
lr = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=10000, decay_rate=0.96, staircase=True)#Learning Rate schedule that uses exponential decay schedule
#Model Compilation
model16.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics=['accuracy'])

In [ ]:
#Printing a summary of the model
model16.summary()

In [ ]:
#Data Augmentation
train_dataset = ImageDataGenerator(rescale=1./255,shear_range=10, zoom_range = 0.2, horizontal_flip = True, 
                            width_shift_range=0.2, fill_mode = 'nearest' )

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#flow from directory
train_dataset = train_dataset.flow_from_directory(train_path, target_size=(224, 224), batch_size=16, class_mode='sparse')

val_dataset = val_datagen.flow_from_directory(val_path, target_size=(224, 224), batch_size=8, class_mode= 'sparse')

test_dataset = val_datagen.flow_from_directory(test_path,target_size=(224, 224),batch_size = 32, class_mode= 'sparse')

In [ ]:
#VGG16 model training with frozen layers
freeze_model = model16.fit(train_dataset,steps_per_epoch=100,epochs=20,validation_data=val_dataset)

In [ ]:
#Get the accuracy of the model
freeze_accuracy=model16.evaluate(test_dataset)

In [ ]:
#Unfreezing the layers
for layer in model16.layers:
    layer.trainable = True

In [ ]:
#Fine tuning the model
lr = 1e-5 #Learning rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr,decay_steps =100000,decay_rate = 0.96, staircase = True)
#Implement early stopping 
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
#Compile the model 
model16.compile(optimizer= tf.keras.optimizers.RMSprop(lr_schedule),loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
#VGG16 with unfrozen layers
unfrozen_model = model16.fit(train_dataset, steps_per_epoch=100,epochs=50, validation_data=val_dataset)


In [ ]:
#Save accuracy scores in an array 
accuracy_Scores = list(unfrozen_model.unfrozen_model['accuracy'])
accuracy_Scores.sort(reverse=True)#Sort the array in descending order

In [ ]:
#Show the accuracy scores of each layer
accuracy_Scores

In [ ]:
#Model Evaluation 
final_Accuracy = model16.evaluate(test_dataset)

In [ ]:
#Output the final accuracy score
final_Accuracy

In [ ]:
#A graph to show the training and validation accuracy graph
%matplotlib inline
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.plot(unfrozen_model.history['accuracy'],label = 'Training Accuracy ')
plt.plot(unfrozen_model.history['val_accuracy'],label = 'Validation Accuracy')
plt.legend()
plt.title('Training V Validation Accuracy')

#Training and Validation Loss
plt.subplot(2,2,2)
plt.plot(unfrozen_model.history['accuracy'],label= 'Training Loss')
plt.plot(unfrozen_model.history['val_accuracy'],label= 'Validation Loss')
plt.legend()
plt.title("Training V Validation Loss")
#Show the plot
plt.show()

In [ ]:
#import metrics#
from sklearn.metrics import confusion_matrix, classification_report
print("Performance Summary of Convolutional Neural Network on test data")

#Show classification report 
print(classification_report(test_dataset.classes, model16.predict(test_dataset).argmax(axis=1)))

#Show confusion matrix
sns.heatmap(confusion_matrix(test_dataset.classes, model16.predict(test_dataset).argmax(axis=1)))
#Change to a sns heatmap
sns.heatmap(classification_report(test_dataset.classes, model16.predict(test_dataset).argmax(axis=1)))

In [ ]:
import keras as keras

In [ ]:
model16 = keras.models.load_model("C:/Users/tedjo/Downloads/XRAY/COVID_Pneumonia_Normal_VGG16_sigmoid.h5")

In [ ]:
import gradio as gr #Gradio to show the model working
def c_n_p(img):#Function which will take in an image
    img = img.reshape((1,64,64,3,1))#Reshape the image to the required shape
    pred = model16.predict(img).tolist()[0]#Receive a model and predict it
    class_names = ['COVID-19','Normal','Pneumonia']#Introduce the three classes used in the training model
    return{class_names[i]:pred[i] for i in range(len(class_names))}#Check the class name and compare it to the prediction

im = gr.inputs.Image(shape=(6144,2),image_mode='L',invert_colors=False,source="upload")#Input image

ui = gr.Interface(fn=c_n_p,inputs=im,outputs=gr.outputs.Label(),title="XRay Classifier",
                article="This is a demo to show the classification of images between three classes: COVID, Pneumonia and Healthy XRay.")#UserInterface

ui.launch(share=True)#Launch the user interface with share, creating a shareable link

**ResNet50**

In [ ]:
from tensorflow.keras.applications import resnet50
from tensorflow.keras.layers import Add, Dense, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

#Input
model50 = resnet50.ResNet50(weights='imagenet',include_top=False,input_shape=(64,64,3))#Load the pretrained model

for layer in model50.layers:
    layer.trainable = False #Freeze the layers
#Output 
output = model50.output
#Pooling layer
output = keras.layers.GlobalAveragePooling2D()(output)#Global average pooling
output = keras.layers.Dense(128,activation='relu')(output)#Dense Layer
predictions = keras.layers.Dense(12,activation='sigmoid')(output)#Dense Layer

model50 = keras.models.Model(inputs=model50.input,outputs=predictions)#Grouping the layers

lr = 0.1#Learning rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr,decay_steps =100000,decay_rate = 0.96, staircase = True)

model50.compile(loss= 'sparse_categorical_crossentropy',optimizer= tf.keras.optimizers.Adam(lr_schedule),metrics=['accuracy'])#Model Compilation

In [ ]:
model50.summary()

In [ ]:
#ResNet50 model training with frozen layers
freeze_model50 = model50.fit(train_dataset,steps_per_epoch=100,epochs=20,validation_data=val_dataset)

In [ ]:
freeze_accuracy50 = model50.evaluate(test_dataset)

In [ ]:
for layers in model50.layers:
    layers.trainable = True#Unfreezing the layers

In [ ]:
lr = 1e-5 #New Learning rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr,decay_steps =100000,decay_rate = 0.96, staircase = True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)#Implement early stopping, where after 5 epochs the model will stop training if there is no improvement
model50.compile(optimizer=tf.keras.optimizers.RMSprop(lr_schedule),loss=keras.losses.SparseCategoricalCrossentropy(),metrics=[keras.metrics.SparseCategoricalAccuracy()])#Compile the new model

In [ ]:
#ResNet50 model training with frozen layers
unfreeze_model50 = model50.fit(train_dataset,steps_per_epoch=100,epochs=50,validation_data=val_dataset)

In [ ]:
unfreeze_accuracy50 = model50.evaluate(test_dataset)#Accuracy of the new model

In [ ]:
model50 = tf.keras.models.save_model(model50,"C:/Users/tedjo/Downloads/XRAY/COVID_Pneumonia_Normal_ResNet_50_sigmoid.h5")

**EfficientNetB3**

In [ ]:
from tensorflow.keras.applications import efficientnet
modelb3 = efficientnet.EfficientNetB3(weights='imagenet',include_top=False,input_shape=(64,64,3))

for i in modelb3.layers:
    i.trainable = False

output = modelb3.output
#The Average Pooling Layer
output = keras.layers.GlobalAveragePooling2D()(output)#Pooling operation for spatial data(Data that directly or indirectly references spatial information)
output = keras.layers.Dense(128, activation='relu')(output)#Dense indicates the number of neurons in the layer, 128 Neurons 
#The classification Layer
predictions = keras.layers.Dense(12, activation='sigmoid')(output)#12 Neurons 
#The final Model
modelb3 = keras.models.Model(inputs=modelb3.input, outputs=predictions)#Group the layers into an object with training and interface features
#Learning rate
lr = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=10000, decay_rate=0.96, staircase=True)#Learning Rate schedule that uses exponential decay schedule
#Model Compilation
modelb3.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics=['accuracy'])

In [ ]:
modelb3.summary()

In [ ]:
freeze_accuracyv3 = modelb3.fit(train_dataset,steps_per_epoch=100,epochs=20,validation_data=val_dataset)

In [ ]:
for layers in modelb3.layers:
    layers.trainable = True

In [ ]:
lr = 1e-5
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr,decay_steps =100000,decay_rate = 0.96, staircase = True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)#Implement early stopping, where after 5 epochs the model will stop training if there is no improvement
modelb3.compile(optimizer=tf.keras.optimizers.RMSprop(lr_schedule),loss=keras.losses.SparseCategoricalCrossentropy(),metrics=[keras.metrics.SparseCategoricalAccuracy()])#Compile the new model

In [ ]:
unfreeze_modelb3 = modelb3.fit(train_dataset,steps_per_epoch=100,epochs=50,validation_data=val_dataset)

In [ ]:
modelb3_accuracy = modelb3.evaluate(test_dataset)

In [ ]:
modelb3 = tf.keras.models.save_model(modelb3,"C:/Users/tedjo/Downloads/XRAY/COVID_Pneumonia_Normal_EfficientNet_B3_sigmoid.h5")

**InceptionV3**

In [ ]:
from tensorflow.keras.applications import inception_v3

modelv3 = inception_v3.InceptionV3(weights='imagenet',include_top=False,input_shape=(224,224,3))

for i in modelv3.layers:
    i.trainable = False

output = modelv3.output

output = keras.layers.GlobalAveragePooling2D()(output)
output = keras.layers.Dense(128,activation='relu')(output)
predictions = keras.layers.Dense(12,activation='sigmoid')(output)

modelv3 = keras.models.Model(inputs=modelv3.input,outputs=predictions)

lr = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr,decay_steps =100000,decay_rate = 0.96, staircase = True)

modelv3.compile(loss= 'sparse_categorical_crossentropy',optimizer= tf.keras.optimizers.Adam(lr_schedule),metrics=['accuracy'])

In [ ]:
modelv3.summary()

In [ ]:
freeze_modelv3 = modelv3.fit(train_dataset,steps_per_epoch=100,epochs=20,validation_data=val_dataset)

In [ ]:
for layers in modelv3.layers:
    layers.trainable = True

In [ ]:
lr = 1e-5
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr,decay_steps =100000,decay_rate = 0.96, staircase = True)
modelv3.compile(optimizer=tf.keras.optimizers.RMSprop(lr_schedule),loss=keras.losses.SparseCategoricalCrossentropy(),metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
unfreeze_accuracyv3 = modelv3.fit(train_dataset,steps_per_epoch=100,epochs=50,validation_data=val_dataset)

In [ ]:
modelv3_accuracy = modelv3.evaluate(test_dataset)

In [ ]:
modelv3 = tf.keras.models.load_model("C:/Users/tedjo/Downloads/XRAY/COVID_Pneumonia_Normal_Inception_V3.h5")

In [ ]:
modelb3 = tf.keras.models.load_model("C:/Users/tedjo/Downloads/XRAY/COVID_Pneumonia_Normal_EfficientNet_B3_sigmoid.h5")

In [ ]:
model50 = tf.keras.models.load_model("C:/Users/tedjo/Downloads/XRAY/COVID_Pneumonia_Normal_ResNet_50.h5")

In [ ]:
modelv3 = tf.keras.models.load_model("C:/Users/tedjo/Downloads/XRAY/COVID_Pneumonia_Normal_Inception_V3_sigmoid.h5")

In [ ]:
#import metrics#
from sklearn.metrics import confusion_matrix, classification_report
print("Performance Summary of VGG16 Network on test data")

#Show classification report
print('Classification Report:')
print(classification_report(test_dataset.classes, model16.predict(test_dataset).argmax(axis=1)))
print('\n')
#Show confusion matrix
print('Confusion Matrix:')
print(confusion_matrix(test_dataset.classes, model16.predict(test_dataset).argmax(axis=1)))

In [ ]:
#import metrics#
from sklearn.metrics import confusion_matrix, classification_report
print("Performance Summary of EfficientNetB3 Network on test data")

#Show classification report
print('Classification Report:')
print(classification_report(test_dataset.classes, modelb3.predict(test_dataset).argmax(axis=1)))
print('\n')
#Show confusion matrix
print('Confusion Matrix:')
print(confusion_matrix(test_dataset.classes, modelb3.predict(test_dataset).argmax(axis=1)))



In [ ]:
#import metrics#
from sklearn.metrics import confusion_matrix, classification_report
print("Performance Summary of ResNet50 Network on test data")

#Show classification report
print('Classification Report:')
print(classification_report(test_dataset.classes, model50.predict(test_dataset).argmax(axis=1)))
print('\n')
#Show confusion matrix
print('Confusion Matrix:')
print(confusion_matrix(test_dataset.classes, model50.predict(test_dataset).argmax(axis=1)))



In [ ]:
#import metrics#
from sklearn.metrics import confusion_matrix, classification_report
print("Performance Summary of InceptionV3 Network on test data")

#Show classification report
print('Classification Report:')
print(classification_report(test_dataset.classes, modelv3.predict(test_dataset).argmax(axis=1)))

print('\n')
#Show confusion matrix
print('Confusion Matrix:')
print(confusion_matrix(test_dataset.classes, modelv3.predict(test_dataset).argmax(axis=1)))

